<a href="https://colab.research.google.com/github/GustavoSouza5900/Truco-Python/blob/main/Truco_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Truco simplesmente
Execute todas as celulas em ordem para deixar tudo certinho

In [ ]:
from random import choice

class Cartas:
    '''Classe simples para criação de cartas e outras coisas
    É só ler o nome das funções'''

    naipes = ['♦️','♠️','♥️', '♣️']
    valores = [4, 5, 6, 7, 8, 9, 10, 1, 2, 3]
    def __init__(self, valor: int, naipe: str, simplificada=''):
        self.valor = valor
        self.naipe = naipe
        self.simp = simplificada
        self.manilha = False
        self.pontos = 0


    @staticmethod
    def gerador_cartas() -> object:
        '''Gera uma carta aleatória'''

        valor = choice(Cartas.valores)
        naipe = choice(Cartas.naipes)
        valor_string = str(valor).replace('8', 'Q').replace('9', 'J').replace('10', 'K').replace('1', 'A')
        simp = f'{valor_string}{naipe}'

        return Cartas(valor, naipe, simp)


    @staticmethod
    def distribuidor(qntd) -> list:
        '''Gera uma lista de cartas aleatórias não repetidas e determina se são manilhas baseando-se na ideia de que a ultima carta gerada é a vira'''
        cartas = []
        simps = []
        while len(cartas) != qntd:
            carta = Cartas.gerador_cartas()
            if carta.simp not in simps:
                cartas.append(carta)
                simps.append(carta.simp)

        Cartas.manilhas(cartas)

        return cartas


    @staticmethod
    def manilhas(cartas):
        '''Recebe uma lista de cartas com a ultima carta sendo a vira e altera o valor de self.manilha dessas cartas'''
        for carta in cartas:
            if (carta.valor == cartas[-1].valor + 1) or (carta.valor == 1 and cartas[-1].valor == 10):
                carta.manilha = True


    @staticmethod
    def combate(cj, cm) -> int:
        '''Define a carta vencedora, retorna 0 se o jogador vencer, 1 se perder e 2 se empachar'''
        #se alguem tiver zap leva na hora
        if cj.manilha and cj.naipe == Cartas.naipes[3]:
            return 0

        if cm.manilha and cm.naipe == Cartas.naipes[3]:
            return 1

        #ninguem tem zap porem se o jogador tiver manilha e a maquina não ele leva
        #se a maquina também tiver manilha comparamos os naipes
        if cj.manilha:
            if cm.manilha == False:
                return 0

            if Cartas.naipes.index(cj.naipe) > Cartas.naipes.index(cm.naipe):
                return 0

        #se o jogador não tem manilha mas a máquina tem ela vence
        if cm.manilha:
            return 1

        #ninguem tem manilha ou zap nesse ponto, simplesmente compramos qual carta é mais forte
        # porem antes é necessario verificar se não vai empatar
        if Cartas.valores.index(cj.valor) == Cartas.valores.index(cm.valor):
            return 2

        if Cartas.valores.index(cj.valor) > Cartas.valores.index(cm.valor):
            return 0

        #se o código chegar até aq a maquina vence em todas as possibilidades
        return 1



In [ ]:
class Jogador:
    '''Classe para ações do jogador, podendo ser um player ou a maquina'''
    def __init__(self, mao=[], nome='Jogador', pontos=0):
        self.mao = mao
        self.nome = nome
        self.pontos = pontos
        # self.trucou = False


    def exibir_mao(self):
        for carta in self.mao:
            print(carta.simp, end=' ')
        print()


    def acao(self) -> object | None:
        '''Retorna e remove a carta escolhida da mão do jogador ou retorna None no caso dele trucar'''

        while True:
            acao = input('1 - jogar carta 1 \n2 - jogar carta 2 \n3 - jogar carta 3 \nação: ')
            print()
            if acao.isnumeric() == False:
                print('Por favor, insira um numero')
                continue

            if int(acao) > len(self.mao):
                print('Você não tem tantas cartas!')
                continue

            match acao:
                case '1':
                    cj = self.mao[0]
                    self.mao.pop(0)

                case '2':
                    cj = self.mao[1]
                    self.mao.pop(1)

                case '3':
                    cj = self.mao[2]
                    self.mao.pop(2)

            return cj


    def calc_pontos(self):
        #calcula pontos de cada carta
        for carta in self.mao:
            if carta.manilha:
                if carta.naipe == Cartas.naipes[3]:
                    carta.pontos = 11
                else:
                    carta.pontos = 10

            else:
                carta.pontos = Cartas.valores.index(carta.valor)


In [ ]:
class Maquina(Jogador):
    def __init__(self, mao=[], nome='máquina', pontos=0):
        super().__init__(mao, nome, pontos)
        self.vez = False


    def decidir(self, cj=False):
        cm = None
        if cj == False:
            if len(self.mao) == 3:
                cm = self.mao[1]
                self.mao.pop(1)
                return cm

            if len(self.mao) == 2 and self.pontos == 0:
                cm = self.mao[1]
                self.mao.pop(1)
                return cm

            cm = self.mao[0]
            self.mao.pop(0)
            return cm

        # define qual carta jogar baseado na carta do oponente
        for pos, carta in enumerate(self.mao):
            if carta.pontos > cj.pontos:
                cm = carta
                self.mao.pop(pos)
                break

        for pos, carta in enumerate(self.mao):
            if carta.pontos == cj.pontos:
                cm = carta
                self.mao.pop(pos)
                break

        if cm == None:
            cm = self.mao[0]
            self.mao.pop(0)
        return cm


In [ ]:
class Truco:
    @staticmethod
    def exibir_placar(jogador, oponente):
        placar = f'Jogo: {jogador.nome} {jogador.pontos} X {oponente.pontos} {oponente.nome}'
        print(f'{placar:^40}')


    @staticmethod
    def draw_line():
        print('-=' * 20)


In [ ]:
jogador = Jogador(nome = input('Escolha seu nome: '))
maquina = Maquina()

while (jogador.pontos < 12) or (maquina.pontos < 12):
    Truco.draw_line()
    Truco.exibir_placar(jogador, maquina)

    cartas = Cartas.distribuidor(7)
    jogador.mao = cartas[:3]
    maquina.mao = cartas[3:6]

    jogador.calc_pontos() # calcula os pontos das cartas, importante pra decisão da "IA" da maquina
    maquina.calc_pontos()

    jogador.mao = sorted(jogador.mao, key=lambda carta: carta.pontos) # Organiza as cartas por ordem da mais fraca pra mais forte
    maquina.mao = sorted(maquina.mao, key=lambda carta: carta.pontos) # não sei usar lambda, copiei e funcionou

    vira = cartas[6]

    primeira = None
    jturno = mturno = 0
    while True:
        Truco.draw_line()
        print(f'rodada: {jogador.nome} {jturno} X {mturno} maquina')
        print(f'vira => {vira.simp}')
        print('Sua mão: ', end='')
        jogador.exibir_mao()
        Truco.draw_line()

        if maquina.vez:
            cm = maquina.decidir()
            print(f'a maquina jogou: {cm.simp}')

            cj = jogador.acao()
            print(f'Você jogou: {cj.simp}')

        else:
            cj = jogador.acao()
            print(f'Você jogou: {cj.simp}')

            cm = maquina.decidir(cj)
            print(f'a maquina jogou: {cm.simp}')

        resultado = Cartas.combate(cj, cm)
        if primeira == None:
            primeira = resultado

        if resultado == 0:
            jturno += 1
            maquina.vez = False
            print('\nVocê ganhou a rodada\n')

        elif resultado == 1:
            mturno += 1
            maquina.vez = True
            print('\nA maquina ganhou a rodada\n')

        else:
            print('\nempatou\n')
            if primeira == None:
                mturno = 1
                jturno = 1

            else:
                if primeira == 0:
                    jturno = 2
                else:
                    mturno = 2

        if jturno >= 2:
            jogador.pontos += 1
            break

        elif mturno >= 2:
            maquina.pontos += 1
            break

print('Obrigado por jogar')
input('\nPressione qualquer tecla para finalizar o programa')


Escolha seu nome: JoA
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
        Jogo: JoA 0 X 0 máquina         
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
rodada: JoA 0 X 0 maquina
vira => 2♥️
Sua mão: 5♣️ K♦️ K♣️ 
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
1 - jogar carta 1 
2 - jogar carta 2 
3 - jogar carta 3 
ação: 2

Você jogou: K♦️
a maquina jogou: A♥️

A maquina ganhou a rodada

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
rodada: JoA 0 X 1 maquina
vira => 2♥️
Sua mão: 5♣️ K♣️ 
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
a maquina jogou: J♣️
1 - jogar carta 1 
2 - jogar carta 2 
3 - jogar carta 3 
ação: 2

Você jogou: K♣️

Você ganhou a rodada

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
rodada: JoA 1 X 1 maquina
vira => 2♥️
Sua mão: 5♣️ 
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
1 - jogar carta 1 
2 - jogar carta 2 
3 - jogar carta 3 
ação: 1

Você jogou: 5♣️
a maquina jogou: 4♥️

Você ganhou a rodada

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
        Jogo: JoA 1 X 0 máquina         
-=-=-=-=-=-=-=-=-

KeyboardInterrupt: Interrupted by user